1. Обучить AE собственной архитектуры на MNIST
1. Обучить VAE собственной архитектуры на MNIST
1. *Обучить VAE c переносом стиля на MNIST (на вход декодеру подавать hidden с таргетным значением, чтобы можно было нарисовать заданную цифру в заданном стиле)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, models
from torchvision.transforms import ToTensor

In [3]:
train_data = datasets.MNIST(
    root='.', train=True, download=True, transform=ToTensor())
test_data = datasets.MNIST(root='.', train=False,
                           download=True, transform=ToTensor())

1. Обучить AE собственной архитектуры на MNIST